<a href="https://colab.research.google.com/github/ucfilho/Raianars_paper_one_revisited/blob/main/After_asked_reviews_one_2022/Testing_CEC2014/F1_to_F30_cec2014_fast_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from scipy.optimize import differential_evolution
import numpy as np
from pandas import read_csv


In [2]:
!git clone https://github.com/ucfilho/functions_cec2014_data
%cd functions_cec2014_data
import F1_to_F27_cec2014
import cec2014_names
import cec2014_shift_rotate

fatal: destination path 'functions_cec2014_data' already exists and is not an empty directory.
/content/functions_cec2014_data


In [3]:
# Note: cec2014 funtions not composed
# F1: elliptic  F2: bent cigar  F3: discus  F4: Rosenbrock
# F5: Ackey   F6: Weierstrass  F7: griewank  F8: rastrigin
# F9: modified Schwefel   F10: Katsuura  F11: happy cat  F12: hgbat
# F13: expanded Griewank   F14:  expanded Scaffer

In [4]:
# F1 Rotated High Conditioned Elliptic Function 
function_select = 'F1' 
Function = F1_to_F27_cec2014.F1 
Shift_Rotate= cec2014_shift_rotate.F1  
problem_size = 10 # possibles 2, 10, 20, 30, 50, 100

# F2 bent cigar 
function_select = 'F2' 
Function = F1_to_F27_cec2014.F2 
Shift_Rotate= cec2014_shift_rotate.F2  
problem_size = 10 # possibles 2, 10, 20, 30, 50, 100

# F3 discus
function_select = 'F3'
Function = F1_to_F27_cec2014.F3
Shift_Rotate= cec2014_shift_rotate.F3  
problem_size = 10 # possibles 2, 10, 20, 30, 50, 100

# F6:Weierstrass
function_select = 'F6'
Function = F1_to_F27_cec2014.F6  
Shift_Rotate= cec2014_shift_rotate.F6  
problem_size = 50 # possibles 2, 10, 20, 30, 50, 100

# F17 to F22 Hybride  F17, F18,
function_select = 'F28' 
Function = F1_to_F27_cec2014.F28
Shift_Rotate= cec2014_shift_rotate.F28 
problem_size = 10 # 


In [5]:
f_matrix = "M_"+function_select[1:]+"_D"+str(problem_size) + ".txt"
shift_data_file="shift_data_"+function_select[1:]+".txt"
F_bias = cec2014_names.BiasValue()[function_select]
type_fun = cec2014_names.TypeFunctions()[function_select]
Function_CEC2014 =function_select + "_" + cec2014_names.NamesFunctions()[function_select]+"_D_"+str(problem_size)

In [6]:
shift_data_file,f_matrix 

('shift_data_28.txt', 'M_28_D10.txt')

In [7]:
!pip install --upgrade --no-cache-dir gdown

In [8]:
# !gdown --id 1yM-cGzp7F35bXZ-f80uoJHzGqAb52CPz
# https://drive.google.com/file/d/1yM-cGzp7F35bXZ-f80uoJHzGqAb52CPz/view?usp=sharing
!gdown --id 1yM-cGzp7F35bXZ-f80uoJHzGqAb52CPz 

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1yM-cGzp7F35bXZ-f80uoJHzGqAb52CPz
To: /content/functions_cec2014_data/cec2014.zip
100% 36.1M/36.1M [00:00<00:00, 281MB/s]


In [9]:
import zipfile
file_name = zipfile.ZipFile('cec2014.zip', 'r')
file_name.extractall()

In [10]:
#!ls

In [11]:
def load_shift_data__(data_file=None):
    data = read_csv(data_file, delimiter='\s+', index_col=False, header=None)
    return data.values.reshape((-1))

def load_matrix_data__(data_file=None):
    data = read_csv(data_file, delimiter='\s+', index_col=False, header=None)
    return data.values

In [12]:
shift_data_a = load_shift_data__(shift_data_file)[:problem_size]
matrix_a = load_matrix_data__(f_matrix)
if (type_fun == 'hybrid'):
  f_shuffle = "shuffle_data_"+function_select[1:]+"_D"+str(problem_size) + ".txt"
  shuffle = (load_shift_data__(f_shuffle)[:problem_size] - np.ones(problem_size)).astype(int)
else:
  shuffle = matrix_a

In [13]:
#!ls

In [14]:
SUPPORT_DIMENSION = [2, 10, 20, 30, 50, 100]
SUPPORT_DIMENSION_2 = [10, 20, 30, 50, 100]

#def Fun(solution,shift_data=shift_data_a, matrix=matrix_a, shuffle):#, bias=F_bias):
def Fun(solution, shift_data ,  matrix, shuffle):#, bias=F_bias):
  if(type_fun == 'shift_rotate'):
    z = Shift_Rotate(solution , shift_data , matrix,shuffle)
    result = Function(z) + F_bias
  elif(type_fun =='hybrid'): # F17 to F22
    z = Shift_Rotate(solution , shift_data , matrix,shuffle)
    result = z + F_bias
  elif(type_fun =='composition'):
    z = Shift_Rotate(solution , shift_data , matrix,shuffle)
    result = z + F_bias
  else: # (type_fun =='extracomposition'):
    if(function_select == 'F28'):
      xichma = np.array([10, 20, 30, 40, 50])
      lamda = np.array([2.5, 10, 2.5, 5e-4, 1e-6])
      bias = np.array([0, 100, 200, 300, 400])
      function_select_new = 'F15'
      Function = F1_to_F27_cec2014.F15
      f_matrix = "M_"+ function_select_new[1:]+"_D"+str(problem_size) + ".txt"
      shift_data_file="shift_data_"+ function_select_new[1:]+".txt"
      # F_bias = cec2014_names.BiasValue()[function_select_01]
      shift_data_a = load_shift_data__(shift_data_file)[:problem_size]
      matrix_a = load_matrix_data__(f_matrix)
      shuffle = matrix_a
      t1 = solution - shift_data_a[0]
      z = Shift_Rotate(solution , shift_data , matrix,shuffle)
      result = Function(z)
      g1 = lamda[0] * result + bias[0]
      w1 = (1.0 / sqrt(sum(t1 ** 2))) * exp(-sum(t1 ** 2) / (2 * problem_size * xichma[0] ** 2))
      t2 = solution - shift_data[1]
      def f11_happy_cat__(solution=None):
        return (abs(sum(solution**2) - len(solution)))**0.25 + (0.5 * sum(solution**2) + sum(solution))/len(solution) + 0.5
      g2 = lamda[1] * f11_happy_cat__(dot(matrix_a[problem_size:2 * problem_size, :], t2)) + bias[1]
      w2 = (1.0 / sqrt(sum(t2 ** 2))) * exp(-sum(t2 ** 2) / (2 * problem_size * xichma[1] ** 2))

      # 3. Rotated Schwefel's Function F11’
      t3 = solution - shift_data[2]
      def f9_modified_schwefel__(solution=None):
        z = solution + 4.209687462275036e+002
        result = 418.9829 * len(solution)
        for i in range(0, len(solution)):
            if z[i] > 500:
                result -= (500 - z[i]%500)*sin(sqrt(abs(500 - z[i]%500))) - (z[i] - 500)**2 / (10000*len(solution))
            elif z[i] < -500:
                result -= (z[i]%500 - 500)*sin(sqrt(abs(z[i]%500 - 500))) - (z[i] + 500)**2 / (10000*len(solution))
            else:
                result -= z[i]*sin(abs(z[i])**0.5)
        return result
      g3 = lamda[2] * f9_modified_schwefel__(dot(matrix[2 * problem_size: 3 * problem_size, :], t3)) + bias[2]
      w3 = (1.0 / sqrt(sum(t3 ** 2))) * exp(-sum(t3 ** 2) / (2 * problem_size * xichma[2] ** 2))

      # 4. Rotated Expanded Scaffer’s F6 Function F16’
      t4 = solution - shift_data[3]
      def f14_expanded_scaffer__(solution=None):
        def __xy__(x, y):
          return 0.5 + (sin(sqrt(x ** 2 + y ** 2)) ** 2 - 0.5) / (1 + 0.001 * (x ** 2 + y ** 2)) ** 2
        result = __xy__(solution[-1], solution[0])
        for i in range(0, len(solution) - 1):
          result += __xy__(solution[i], solution[i + 1])
        return result
      g4 = lamda[3] * f14_expanded_scaffer__(dot(matrix[3 * problem_size: 4 * problem_size, :], t4)) + bias[3]
      w4 = (1.0 / sqrt(sum(t4 ** 2))) * exp(-sum(t4 ** 2) / (2 * problem_size * xichma[3] ** 2))

      # 5. Rotated High Conditioned Elliptic Function F1’
      t5 = solution - shift_data[4]
      def f1_elliptic__(solution=None):
        result = 0
        for i in range(len(solution)):
          result += (10**6)**(i/(len(solution)-1)) * solution[i]**2
        return result
      g5 = lamda[4] * f1_elliptic__(dot(matrix[4 * problem_size:, :], t5)) + bias[4]
      w5 = (1.0 / sqrt(sum(t5 ** 2))) * exp(-sum(t5 ** 2) / (2 * problem_size * xichma[4] ** 2))
      sw = sum([w1, w2, w3, w4, w5])
      result = (w1 * g1 + w2 * g2 + w3 * g3 + w4 * g4 + w5 * g5) / sw
  return result


In [15]:
solution = np.random.uniform(0, 1,problem_size)
resp = Fun(solution, shift_data = shift_data_a, matrix = matrix_a, shuffle = shuffle)
print(resp)

IndexError: ignored

In [ ]:
bounds_test = int(problem_size)*[(-100,100)]
result = differential_evolution(Fun, bounds_test)

In [ ]:
result.x, result.fun, Function_CEC2014